### Twitter dataset sentiment analysis

#### Dataset από kaggle (περιέχει 1.6 εκατομμύρια tweets).

#### Η μεταβλητή εξόδου έχει 3 τιμές: 0 (negative), 2 (neutral) και 4 (positive).

### Κάνουμε import τις απαραίτητες βιβλιοθήκες

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


import joblib
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'wordcloud'

### Διαβάζουμε και βλέπουμε το dataset

In [ ]:
df = pd.read_csv("./data/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", names= ["target", "ids", "date", "flag", "user", "text"])
df

### Δημιουργούμε plots για να οπτικοποιήσουμε τα δεδομένα μας

In [3]:
# Plot 1: Distribution of Sentiment Classes
plt.figure(figsize=(8, 6))
sns.countplot(df['target'])
plt.title('Distribution of Sentiment Classes')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plot 2: Number of Tweets Over Time
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

plt.figure(figsize=(12, 6))
sns.lineplot(data=df.groupby(['year', 'month'])['ids'].count().reset_index(), x='year', y='ids')
plt.title('Number of Tweets Over Time')
plt.xlabel('Year')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()



In [ ]:
# Plot 3: User Distribution
user_counts = df['user'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=user_counts.values, y=user_counts.index, palette='viridis')
plt.title('Top 10 Users by Tweet Count')
plt.xlabel('Tweet Count')
plt.ylabel('User')
plt.show()

In [ ]:
# Plot 4: Tweet Length Distribution
df['tweet_length'] = df['text'].apply(len)

plt.figure(figsize=(8, 6))
sns.histplot(data=df, x='tweet_length', bins=30, kde=True, color='skyblue')
plt.title('Distribution of Tweet Length')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')
plt.show()

### Φιλτράρουμε τα θετικά φορτισμένα tweets επιλέγοντας τις γραμμές όπου το target έχει τιμή 4.

In [ ]:
# Filter the positive sentiment tweets (where target == 4)
positive_tweets = df[df['target'] == 4]['text'].str.cat(sep=' ')

plt.figure(figsize=(10, 8))
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(positive_tweets)
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Positive Sentiment Tweets')
plt.axis('off')
plt.show()

### Αλλάζουμε το πλήθος των δειγμάτων κάθε κλάσης

In [ ]:
samples_per_class = 50000
balanced_df = df.groupby('target').apply(lambda x: x.sample(samples_per_class)).reset_index(drop=True)

### Χωρίζουμε το dataset σε training και test sets

### Επίσης χρησιμοποιούμε την τεχνική tf-idf για vectorization των κειμένων

In [ ]:
X = balanced_df['text']
y = balanced_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

### Ορίζουμε και εκπαιδεύουμε μοντέλα μηχανικής μάθησης

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(),
    'SVC': SVC(),
    'LGBM': LGBMClassifier(),
#     'XGBoost': XGBClassifier()
}

for name, model in models.items():
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)
    accuracy = accuracy_score(y_test, predictions)
    print(f'{name} Accuracy: {accuracy}')

### Σώζουμε τα εκπαιδευμένα μοντέλα σε αρχεία pickle και δημιουργούμε confusion matrices

In [ ]:
saved_models = {}

# Save the vectorizer
vectorizer_filename = 'vectorizer.pkl'
joblib.dump(vectorizer, vectorizer_filename)

# Print the filenames of the saved models and vectorizer
print("Saved models:")
print(saved_models)
print("Saved vectorizer:", vectorizer_filename)

#Saving the models...
for name, model in models.items():
    # Save the trained model
    model_filename = f'{name}_model.pkl'
    joblib.dump(model, model_filename)
    saved_models[name] = model_filename
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_test, predictions)
    
    print(f'{name} Accuracy: {accuracy}')
    print(f'{name} Confusion Matrix:')
    print(cm)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix for {name}')
    plt.show()